In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI

### 1 - Loading the LLM

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2 - Defining the output parsers

#### Cluster

In [3]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

cluster_response_schemas = [
    ResponseSchema(name="prompt", description="prompt given by the user"),
    ResponseSchema(
        name="cluster",
        description="cluster in which the prompt is classified",
    ),
]
cluster_output_parser = StructuredOutputParser.from_response_schemas(
    cluster_response_schemas
)
cluster_format_instructions = cluster_output_parser.get_format_instructions()

#### Sub Class

In [4]:
subclass_response_schemas = [
    ResponseSchema(name="prompt", description="prompt given by the user"),
    ResponseSchema(
        name="cluster",
        description="Category in which the prompt is classified",
    ),
    ResponseSchema(
        name="Sub Class",
        description="Sub category in which the prompt is classified into",
    ),
]
subclass_output_parser = StructuredOutputParser.from_response_schemas(
    subclass_response_schemas
)
subclass_format_instructions = subclass_output_parser.get_format_instructions()

### 3 - Initializing the Chains

In [5]:
from prompt_templates import (
    cluster_prompt_template,
    communication_prompt_template,
    music_prompt_template,
    programming_prompt_template,
    business_prompt_template,
)

In [6]:
cluster_prompt = PromptTemplate(
    template=cluster_prompt_template,
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": cluster_format_instructions},
)

cluster_chain = cluster_prompt | llm | cluster_output_parser

In [7]:
prompt_templates = [
    communication_prompt_template,
    music_prompt_template,
    programming_prompt_template,
    business_prompt_template,
]
categories = [
    "Communication",
    "Music and Audio",
    "Programming and Development",
    "Business and Productivity",
]

subclass_chains = {}

for i, prompt_template in enumerate(prompt_templates):

    subclass_prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["user_prompt"],
        partial_variables={"format_instructions": subclass_format_instructions},
    )
    chain = subclass_prompt | llm | subclass_output_parser
    subclass_chains[f"{categories[i]}"] = chain

In [8]:
subclass_chains.keys()

dict_keys(['Communication', 'Music and Audio', 'Programming and Development', 'Business and Productivity'])

In [9]:
subclass_chains["Business and Productivity"]

PromptTemplate(input_variables=['user_prompt'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"prompt": string  // prompt given by the user\n\t"cluster": string  // Category in which the prompt is classified\n\t"Sub Class": string  // Sub category in which the prompt is classified into\n}\n```'}, template='You are a helpful and accurate AI assistant specializing in business and productivity tasks.\n\nHere are the sub-categories within Business and Productivity:\n- Presentation Creation\n- Email Generation\n\nYou will be provided with a user request related to Business and Productivity. Your task is to determine the most specific sub-category that applies to the request. \nDo not use any other sub-categories. If the user request does not match with any of the sub categories, classify it as "General Model".\n\nExamples:\nPresentation Creati

### 4 - Evaluating chains

In [10]:
prompt = "How to cook pasta?"
cluster_result = cluster_chain.invoke({"user_prompt": prompt})

In [11]:
cluster_result

{'prompt': 'How to cook pasta?', 'cluster': 'General Model'}

In [12]:
subclass_result = subclass_chains["Business and Productivity"].invoke(
    {"user_prompt": prompt}
)
subclass_result

{'prompt': 'How to cook pasta?',
 'cluster': 'Business and Productivity',
 'Sub Class': 'General Model'}

### 5 - Classfication Pipeline

In [18]:
def classify_request(user_prompt):

    cluster_result = cluster_chain.invoke({"user_prompt": user_prompt})
    cluster = cluster_result["cluster"]
    if cluster != "General Model":
        sub_classification_result = subclass_chains[cluster].invoke(
            {"user_prompt": user_prompt}
        )
        return sub_classification_result
    else:
        cluster_result["Sub Class"] = "N/A"
        return cluster_result

In [19]:
classify_request("How to write a mail to my manager")

{'prompt': 'How to write a mail to my manager',
 'cluster': 'Business and Productivity',
 'Sub Class': 'Email Generation'}

In [20]:
classify_request("How to handle depression?")

{'prompt': 'How to handle depression?',
 'cluster': 'General Model',
 'Sub Class': 'N/A'}

### 6 - Evaluation Metrics

In [24]:
import pandas as pd

df = pd.read_csv("../data/raw/prompts_v1.csv")
sampled_df = (
    df.groupby(["cluster", "sub_class"])
    .apply(lambda x: x.sample(min(3, len(x)), random_state=42))
    .reset_index(drop=True)
)

In [26]:
sampled_df

,prompt,cluster,sub_class
0,Compose an email to express appreciation to a ...,Business and Productivity,Email Generation
1,Write an email to request a referral from a sa...,Business and Productivity,Email Generation
2,Write an email to remind clients about an upco...,Business and Productivity,Email Generation
3,What are some creative ways to present case st...,Business and Productivity,Presentation Creation
4,What are some innovative presentation formats ...,Business and Productivity,Presentation Creation
5,What strategies can I use to reduce anxiety be...,Business and Productivity,Presentation Creation
6,How do I create a personality for my chatbot?,Communication,Chatbots and Virtual Assistants
7,What are the differences between rule-based an...,Communication,Chatbots and Virtual Assistants
8,What should I include in a chatbot's FAQ section?,Communication,Chatbots and Virtual Assistants
9,What are some ways to build rapport quickly in...,Communication,Conversation


In [28]:
true_clusters = []
predicted_clusters = []
true_sub_classes = []
predicted_sub_classes = []

for index, row in sampled_df.iterrows():
    prompt = row["prompt"]
    actual_cluster = row["cluster"]
    actual_sub_class = row["sub_class"]

    prediction = classify_request(prompt)

    predicted_cluster = prediction["cluster"]
    predicted_sub_class = prediction["Sub Class"]

    true_clusters.append(actual_cluster)
    predicted_clusters.append(predicted_cluster)
    true_sub_classes.append(actual_sub_class)
    predicted_sub_classes.append(predicted_sub_class)

#### Cluster Metrics

In [29]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [30]:
# Calculate accuracy, F1 score and confusion matrix for cluster
cluster_accuracy = accuracy_score(true_clusters, predicted_clusters)
cluster_f1 = f1_score(true_clusters, predicted_clusters, average="weighted")
cluster_confusion_matrix = confusion_matrix(true_clusters, predicted_clusters)

print(f"Cluster Accuracy: {cluster_accuracy * 100:.2f}%")
print(f"Cluster F1 Score: {cluster_f1:.2f}")
print("Cluster Confusion Matrix:")
print(cluster_confusion_matrix)

Cluster Accuracy: 86.67%
Cluster F1 Score: 0.87
Cluster Confusion Matrix:
[[4 2 0 0]
 [0 9 0 0]
 [0 2 7 0]
 [0 0 0 6]]


In [31]:
sub_class_accuracy = accuracy_score(true_sub_classes, predicted_sub_classes)
sub_class_f1 = f1_score(true_sub_classes, predicted_sub_classes, average="weighted")
sub_class_confusion_matrix = confusion_matrix(true_sub_classes, predicted_sub_classes)

print(f"Sub Class Accuracy: {sub_class_accuracy * 100:.2f}%")
print(f"Sub Class F1 Score: {sub_class_f1:.2f}")
print("Sub Class Confusion Matrix:")
print(sub_class_confusion_matrix)

Sub Class Accuracy: 83.33%
Sub Class F1 Score: 0.85
Sub Class Confusion Matrix:
[[3 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 1 0 0 0 0]
 [0 0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 1 0 0 2 0]
 [0 0 0 0 0 2 0 0 0 0 1]]
